# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Lora Model
* Model: Roberta
* Evaluation approach: Used accuracy metric on the test dataset. The evaluation process includes assessing the base model, the fully fine-tuned model, and the PEFT fine-tuned model using the test dataset and comparing their performance.
* Fine-tuning dataset: Deysi/sentences-and-emotions

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install transformers datasets scikit-learn torch peft

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import torch
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, RobertaTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForCausalLM
from datasets import load_dataset, ClassLabel
from peft import LoraConfig, get_peft_model

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Tokenize the dataset
def tokenize_function(examples):
    token=tokenizer(examples['utterance'], truncation=True, padding='longest')
    return token

# Convert string labels to numerical labels
def label_to_id(example):
    label = example["emotion"]
    return {"label": label_to_id_map[label]}

# Configure base model
model_name = 'roberta-base'

# Load the Tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Load the dataset
dataset = load_dataset("Deysi/sentences-and-emotions") 

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["utterance"])

class_names = list(dataset['train'].unique('emotion'))
label_to_id_map = {label: i for i, label in enumerate(class_names)}
id_to_label_map = {i: label for label, i in label_to_id_map.items()}

tokenized_datasets = tokenized_datasets.map(label_to_id, batched=False)

In [3]:
# # Load the Tokenizer
train_dataset = tokenized_datasets["train"].with_format("torch")
eval_dataset = tokenized_datasets["valid"].with_format("torch")
test_dataset = tokenized_datasets["test"].with_format("torch")

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(class_names))
model.config.id2label = id_to_label_map
model.config.label2id = label_to_id_map

data_collator =  DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
    num_train_epochs=1,
#     weight_decay=0.01,
    logging_dir="./logs",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

eval_result = trainer.evaluate()
print(eval_result)


{'eval_loss': 2.2257132530212402, 'eval_accuracy': 0.036511156186612576, 'eval_precision': 0.001333064526083218, 'eval_recall': 0.036511156186612576, 'eval_f1': 0.0025722145258670307, 'eval_runtime': 3.2054, 'eval_samples_per_second': 153.802, 'eval_steps_per_second': 19.342}


/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
!pip install transformers datasets scikit-learn torch peft

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import evaluate
from tqdm import tqdm

# Load the dataset
dataset = load_dataset("Deysi/sentences-and-emotions")
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Adds accuracy, precision, recall, and f1 to output table
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["utterance"], truncation=True, padding="max_length")

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["utterance"])

# Label mapping
class_names = tokenized_datasets["train"].unique("emotion")
label_to_id_map = {label: i for i, label in enumerate(class_names)}
id_to_label_map = {i: label for label, i in label_to_id_map.items()}

# Update labels in the dataset
def label_to_id(example):
    return {"labels": label_to_id_map[example["emotion"]]}

tokenized_datasets = tokenized_datasets.map(label_to_id, batched=False, remove_columns=["emotion"])

# Save paths
full_finetuned_model_path = "./roberta-full-finetuned"
peft_model_name = './roberta-peft'

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=4e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=3,
    metric_for_best_model="eval_loss"
)

# Trainer function
def get_trainer(model):
    return Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"].with_format("torch"),
        eval_dataset=tokenized_datasets["test"].with_format("torch"),
        compute_metrics=compute_metrics,
        data_collator=data_collator
    )

# Full fine-tuning
full_finetuning_trainer = get_trainer(
    AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(class_names), id2label=id_to_label_map, label2id=label_to_id_map)
)
print("Full fine-tuning")
full_finetuning_trainer.train()
full_finetuning_trainer.model.save_pretrained(full_finetuned_model_path)

# Save the tokenizer for the fully fine-tuned model
tokenizer.save_pretrained(full_finetuned_model_path)

# PEFT model training
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=4, lora_alpha=8, lora_dropout=0.3)
peft_model = get_peft_model(
    AutoModelForSequenceClassification.from_pretrained(full_finetuned_model_path, num_labels=len(class_names), id2label=id_to_label_map, label2id=label_to_id_map),
    peft_config
)
print("PEFT Model")
peft_model.print_trainable_parameters()
peft_lora_finetuning_trainer = get_trainer(peft_model)
peft_lora_finetuning_trainer.train()

# Save the PEFT fine-tuned model
peft_model.save_pretrained(peft_model_name)

# Save the tokenizer
tokenizer.save_pretrained("./roberta-tokenizer")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Full fine-tuning


/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.loca

PEFT Model
trainable params: 1,344,020 || all params: 125,399,060 || trainable%: 1.0717943180754306


/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
!pip install transformers datasets scikit-learn torch
!pip install evaluate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import AutoPeftModelForSequenceClassification
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("./roberta-tokenizer")
inference_model = AutoPeftModelForSequenceClassification.from_pretrained("./roberta-peft")

# Prepare label maps
id_to_label_map = {
    0: "neutral",
    1: "surprise",
    2: "anger",
    3: "disgust",
    4: "sadness",
    5: "happiness",
    6: "excited",
    7: "sad",
    8: "fear",
    9: "happines"
}

# Evaluation function
def evaluate_model(model, dataset):
    metric = evaluate.load('accuracy')
    eval_dataloader = DataLoader(dataset, batch_size=8, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        references = batch["labels"]
        metric.add_batch(predictions=predictions, references=references)
    eval_metric = metric.compute()
    print(eval_metric)

In [ ]:
# Evaluate models
print("Evaluate Non-Fine Tuned Model")
evaluate_model(AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=len(id_to_label_map), id2label=id_to_label_map), tokenized_datasets["test"])

print("Evaluate PEFT Model")
evaluate_model(AutoModelForSequenceClassification.from_pretrained(full_finetuned_model_path, num_labels=len(id_to_label_map), id2label=id_to_label_map), tokenized_datasets["test"])

print("Evaluate Fully Fine-Tuned Model")
evaluate_model(inference_model, tokenized_datasets["test"])